In [14]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "mel_images_1_17.csv"
dff = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 96
width = 216
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr = []
main_labels = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in dff.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr.append(sequence_of_frames)
        main_labels.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr))
print("Number of label:", len(main_labels))

Number of sequences: 748
Number of label: 748


In [15]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "mel_images_18_24.csv"
dff = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 96
width = 216
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr_test = []
main_labels_test = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in dff.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr_test.append(sequence_of_frames)
        main_labels_test.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr_test))
print("Number of label:", len(main_labels_test))

Number of sequences: 264
Number of label: 264


In [16]:
main_arr_test=np.array(main_arr_test)
main_labels_test=np.array(main_labels_test)

In [10]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape
from tensorflow.keras.layers import TimeDistributed, BatchNormalization

# Define input shape and number of classes
input_frames = 12
height = 96
width = 216
channels = 3
num_classes = 6
input_shape = (input_frames, height, width, channels)

def create_crnn_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Flatten()))

    # Fully connected layers
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))

    # LSTM layer
    model.add(LSTM(256, return_sequences=False))

    # Fully connected layer after LSTM
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create the CRNN model
model = create_crnn_model(input_shape, num_classes)
# Convert your data to numpy arrays if needed
x_train_np = np.array(main_arr)
y_train_np = np.array(main_labels)

main_arr = np.array(main_arr)
main_labels = np.array(main_labels)


# Split the data using KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in kf.split(main_arr):
    x_train, x_val = main_arr[train_index], main_arr[val_index]
    y_train, y_val = main_labels[train_index], main_labels[val_index]

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=20, batch_size=12, verbose=1)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_val, y_val)
    cv_scores.append(test_accuracy)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

# Evaluate the model on the test set
#test_loss, test_accuracy = model.evaluate(x_test, y_test)
#print("Test Loss:", test_loss)
#print("Test Accuracy:", test_accuracy)

# Make predictions
#predictions = model.predict(x_test)
#print("The predictions are:", predictions)

model.save('CRNN_model.h5')
print("the model saved successfully")


Epoch 1/20
50/50 [==============================] - 99s 2s/step - loss: 1.7000 - accuracy: 0.3729
Epoch 2/20
50/50 [==============================] - 84s 2s/step - loss: 1.2655 - accuracy: 0.5234
Epoch 3/20
50/50 [==============================] - 86s 2s/step - loss: 1.0113 - accuracy: 0.6605
Epoch 4/20
50/50 [==============================] - 84s 2s/step - loss: 0.6928 - accuracy: 0.7408
Epoch 5/20
50/50 [==============================] - 89s 2s/step - loss: 0.5768 - accuracy: 0.7776
Epoch 6/20
50/50 [==============================] - 89s 2s/step - loss: 0.3785 - accuracy: 0.8579
Epoch 7/20
50/50 [==============================] - 88s 2s/step - loss: 0.2737 - accuracy: 0.9047
Epoch 8/20
50/50 [==============================] - 86s 2s/step - loss: 0.2982 - accuracy: 0.9097
Epoch 9/20
50/50 [==============================] - 89s 2s/step - loss: 0.3477 - accuracy: 0.8779
Epoch 10/20
50/50 [==============================] - 85s 2s/step - loss: 0.2498 - accuracy: 0.9097
Epoch 11/20
50/50 [

C:\Users\samatha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


the model saved successfully


In [12]:
test_loss, test_accuracy = model.evaluate(main_arr_test, main_labels_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
predictions = model.predict(main_arr_test)
print("The predictions are:", predictions)

9/9 [==============================] - 30s 2s/step - loss: 1.1937 - accuracy: 0.7652
Test Loss: 1.1936988830566406
Test Accuracy: 0.7651515007019043
9/9 [==============================] - 21s 2s/step
The predictions are: [[9.55049098e-01 1.25555025e-05 1.55764588e-04 1.62051569e-04
  8.35986924e-04 4.37845699e-02]
 [7.49100924e-01 3.46109809e-05 7.01358385e-06 4.00692603e-04
  4.88324964e-04 2.49968484e-01]
 [4.34869295e-03 3.64386051e-06 2.33789269e-05 4.38047355e-05
  1.70626066e-04 9.95409787e-01]
 ...
 [3.33427684e-04 1.30450411e-04 9.02062766e-06 2.22181848e-06
  7.52325786e-06 9.99517322e-01]
 [3.39571270e-05 6.43545127e-06 1.19784299e-05 1.13074272e-07
  1.77526340e-06 9.99945760e-01]
 [7.38882343e-04 4.89311409e-04 8.13653506e-03 3.83527049e-06
  1.25723658e-04 9.90505755e-01]]


In [13]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('CRNN_model.h5')

# Now you can use the loaded model for prediction or further training
# Assuming you have new data stored in a variable called 'new_data'
#predictions = loaded_model.predict(new_data)

# Use the predictions as needed
loaded_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_42 (TimeD  (None, 12, 94, 214, 64)   1792      
 istributed)                                                     
                                                                 
 time_distributed_43 (TimeD  (None, 12, 94, 214, 64)   256       
 istributed)                                                     
                                                                 
 time_distributed_44 (TimeD  (None, 12, 47, 107, 64)   0         
 istributed)                                                     
                                                                 
 time_distributed_45 (TimeD  (None, 12, 45, 105, 64)   36928     
 istributed)                                                     
                                                                 
 time_distributed_46 (TimeD  (None, 12, 45, 105, 64)  